# Set up (Run these first)

In [1]:
from pathlib import Path
import os
from itertools import product
from typing import Dict, Tuple, List

import numpy as np
import polars as pl
from foreach import foreach

from MITRotor.ReferenceTurbines import IEA10MW
from UnifiedMomentumModel.Utilities.FixedPointIteration import fixedpointiteration
from MITRotor.BEM import BEM, BEMSolution


pl.Config.set_tbl_cols(20)


rootdir = Path(os.getcwd()).parent
cachedir = rootdir / "cache"
cachedir.mkdir(exist_ok=True, parents=True)

PITCHES = np.deg2rad(np.arange(-15, 30, 1))
TSRS = np.arange(0, 20, 0.25)
YAWS = np.deg2rad(np.arange(0, 51, 2.5))

# For control trajectories
CTPRIMES = np.arange(0.1, 5, 0.2)

bem = BEM(IEA15MW())


## Helper functions

In [2]:
# Output


def extract_output(sol: BEMSolution) -> Dict[str, float]:
    """
    Extract the output from a bem object and return a consistent dictionary.
    - Returns essential values for output dataframe.
    - If bem is not converged, returns data with nans.
    """
    if sol.converged:
        return dict(
            pitch=np.round(np.rad2deg(sol.pitch), 4),
            tsr=sol.tsr,
            yaw=np.round(np.rad2deg(sol.yaw), 4),
            Cp=sol.Cp(),
            Ct=sol.Ct(),
            Ctprime=sol.Ctprime(),
            a=sol.a(),
            u4=sol.u4(),
            v4=sol.v4(),
        )
    else:
        return dict(
            pitch=np.round(np.rad2deg(sol.pitch), 4),
            tsr=sol.tsr,
            yaw=np.round(np.rad2deg(sol.yaw), 4),
            Cp=np.nan,
            Ct=np.nan,
            Ctprime=np.nan,
            a=np.nan,
            u4=np.nan,
            v4=np.nan,
        )

In [3]:
# Parallel looper


def generate_partial(func, params: list, data_type: str, fn_out: Path, parallel=True):
    """
    Generates data from BEM code by doing the following:
    - Runs input function in parallel for all parameter sets listed in params.
    - Saves data in dataframe. Adds the data_type column.
    - Saves dataframe to fn_out as csv.
    """
    out = foreach(func, params, parallel=parallel)

    df = pl.from_dicts(out).with_columns(pl.lit(data_type).alias("data_type"))

    df.write_csv(fn_out)
    print(f"{data_type} data written to {fn_out}")

    return df

## Calculate zero-yaw optimum

In [6]:
from BEM_root_finding import find_optimal_setpoint

PITCH_OPT, TSR_OPT, _ = find_optimal_setpoint(bem, yaw=0)
CP_OPT = bem(PITCH_OPT, TSR_OPT, 0).Cp()

print(f"{np.rad2deg(PITCH_OPT)=:2.4f}")
print(f"{TSR_OPT=:2.4f}")
print(f"{CP_OPT=:2.4f}")

np.rad2deg(PITCH_OPT)=-1.4355
TSR_OPT=8.9627
CP_OPT=0.5078


# Data Generation (each cell can be run individually)

### Contour
Loop over pitch, tsr and yaw.

In [7]:
fn_out = cachedir / "partial_pitch_tsr_surface.csv"
params = list(product(PITCHES, TSRS, YAWS))


def contour_func(x):
    pitch, tsr, yaw = x
    sol = bem(pitch, tsr, yaw)
    return extract_output(sol)


generate_partial(contour_func, params, "contour", fn_out)

/home/jaime/Repositories/MITRotor/MITRotor/BEM.py:352: RuntimeWarning: divide by zero encountered in true_divide
  aero_props.solidity / (4 * np.maximum(geom.mu, 0.1) ** 2 * tsr * (1 - aero_props.an) * np.cos(yaw)) * tangential_integral
/home/jaime/Repositories/MITRotor/MITRotor/BEM.py:352: RuntimeWarning: divide by zero encountered in true_divide
  aero_props.solidity / (4 * np.maximum(geom.mu, 0.1) ** 2 * tsr * (1 - aero_props.an) * np.cos(yaw)) * tangential_integral
/home/jaime/Repositories/MITRotor/MITRotor/BEM.py:352: RuntimeWarning: divide by zero encountered in true_divide
  aero_props.solidity / (4 * np.maximum(geom.mu, 0.1) ** 2 * tsr * (1 - aero_props.an) * np.cos(yaw)) * tangential_integral
/home/jaime/Repositories/MITRotor/MITRotor/BEM.py:352: RuntimeWarning: divide by zero encountered in true_divide
  aero_props.solidity / (4 * np.maximum(geom.mu, 0.1) ** 2 * tsr * (1 - aero_props.an) * np.cos(yaw)) * tangential_integral
/home/jaime/Repositories/MITRotor/MITRotor/BEM.py:35

KeyboardInterrupt: 

### Global Optimal
Loop over yaw

In [ ]:
fn_out = cachedir / "partial_optimal_setpoints.csv"
params = YAWS


def func(x):
    yaw = x
    setpoint = find_optimal_setpoint(bem, yaw=yaw)
    sol = bem(*setpoint)
    return extract_output(sol)


generate_partial(func, params, "optimal", fn_out)

100%|██████████| 21/21 [00:03<00:00,  5.52it/s]

optimal data written to /home/jaime/Repositories/Torque2024_AD_vs_BEM/cache/partial_optimal_setpoints.csv


### Constant $\lambda$
Loop over yaw

In [9]:
fn_out = cachedir / "partial_constant_tsr.csv"
params = YAWS


def func(x):
    yaw = x
    sol = bem(PITCH_OPT, TSR_OPT, yaw)
    return extract_output(sol)


generate_partial(func, params, "constant_tsr", fn_out)

100%|██████████| 21/21 [00:00<00:00, 3108.10it/s]

constant_tsr data written to /home/jaime/Repositories/Torque2024_AD_vs_BEM/cache/partial_constant_tsr.csv


### $K-\Omega^2$
Loop over yaw

In [6]:
fn_out = cachedir / "partial_k-omega_optimal.csv"
params = YAWS

CP_ON_TSR3 = CP_OPT / TSR_OPT**3


class KOmega:
    def initial_guess(yaw):
        return [7]

    def residual(x, yaw):
        tsr = x[0]
        return np.cbrt(bem(PITCH_OPT, tsr, yaw).Cp() / CP_ON_TSR3) - tsr

    def post_process(sol, yaw):
        return extract_output(sol)


def func(x):
    yaw = x

    def torque_control_residual(tsr):
        sol = bem.solve(PITCH_OPT, tsr, yaw)

        if sol.converged:
            return np.cbrt(sol.Cp() / CP_ON_TSR3) - tsr

    converged, tsr = fixedpointiteration(torque_control_residual, 7, relax=0.6, maxiter=200)
    sol = bem.solve(PITCH_OPT, tsr, yaw)

    return extract_output(sol)


generate_partial(func, params, "k-omega", fn_out)

100%|██████████| 21/21 [00:00<00:00, 45.03it/s]

k-omega data written to /home/jaime/Repositories/Torque2024_AD_vs_BEM/cache/partial_k-omega_optimal.csv


pitch,tsr,yaw,Cp,Ct,Ctprime,a,u4,v4,data_type
f64,f64,f64,f64,f64,f64,f64,f64,f64,str
-2.6079,8.477629,0.0,0.52844,0.860062,1.822063,0.312958,0.374083,-0.0,"""k-omega"""
-2.6079,8.474002,2.5,0.527762,0.859315,1.821991,0.312589,0.374963,-0.009371,"""k-omega"""
-2.6079,8.463194,5.0,0.525745,0.857119,1.822054,0.311513,0.377534,-0.018676,"""k-omega"""
-2.6079,8.445163,7.5,0.522392,0.853493,1.822447,0.309754,0.381747,-0.027851,"""k-omega"""
-2.6079,8.419761,10.0,0.517692,0.848419,1.823201,0.307314,0.387588,-0.036832,"""k-omega"""
-2.6079,8.386747,12.5,0.511627,0.841855,1.824271,0.304187,0.395056,-0.045553,"""k-omega"""
-2.6079,8.345852,15.0,0.504179,0.833761,1.825662,0.300373,0.404139,-0.053948,"""k-omega"""
-2.6079,8.296746,17.5,0.495331,0.824093,1.82741,0.295874,0.414812,-0.061952,"""k-omega"""
-2.6079,8.239037,20.0,0.485067,0.812805,1.829565,0.290694,0.427042,-0.069499,"""k-omega"""


### $C_T'$ minimizing trajectory

In [12]:
from src.BEM_root_finding import find_power_maximising_Ctprime_setpoints

fn_out = cachedir / "partial_setpoint_trajectories.csv"
params = list(product(YAWS, CTPRIMES))


def func(x):
    yaw, ctprime = x
    setpoint = find_power_maximising_Ctprime_setpoints(bem, ctprime, yaw=yaw)
    sol = bem.solve(*setpoint)

    return extract_output(sol)


df_ctprime_min_traj = generate_partial(func, params, "Ctprime_trajectory", fn_out)

100%|██████████| 525/525 [08:40<00:00,  1.01it/s]

Ctprime_trajectory data written to /home/jaime/Repositories/Torque2024_AD_vs_BEM/cache/partial_setpoint_trajectories.csv


### zero-yaw $C_T'$ minimizing trajectory

In [21]:
from src.BEM_root_finding import find_power_maximising_Ctprime_setpoints

# Load Ctprime minimising trajectory (from previous block)
fn_previous = cachedir / "partial_setpoint_trajectories.csv"
df = (
    pl.read_csv(fn_previous)
    .filter(pl.col("data_type") == "Ctprime_trajectory")
    .filter(pl.col("yaw") == 0)
)

fn_out = cachedir / "partial_setpoint_zero_yaw_ctprime_minimizing_trajectories.csv"
params = list(product(YAWS, CTPRIMES))


def func(x):
    yaw, ctprime = x

    # pitch, tsr, _ = find_power_maximising_Ctprime_setpoints(bem, ctprime, yaw=0)
    pitch, tsr = (
        df.filter(pl.col("Ctprime") - ctprime < 0.001)
        .select("pitch", "tsr")
        .to_numpy()[0]
    )

    pitch = np.deg2rad(pitch)
    sol = bem.solve(pitch, tsr, yaw)

    return extract_output(sol)


generate_partial(func, params, "zero_yaw_Ctprime_trajectory", fn_out, parallel=False)

100%|██████████| 525/525 [00:02<00:00, 209.03it/s]

zero_yaw_Ctprime_trajectory data written to /home/jaime/Repositories/Torque2024_AD_vs_BEM/cache/partial_setpoint_zero_yaw_ctprime_minimizing_trajectories.csv


# Join DataFrames

In [22]:
files = list(cachedir.glob("partial*"))

df = pl.concat([pl.read_csv(fn) for fn in files])
print(df)
df.write_csv(cachedir / "tsr_pitch_yaw.csv")

shape: (76_713, 10)
┌─────────┬──────────┬──────┬──────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│ pitch   ┆ tsr      ┆ yaw  ┆ Cp       ┆ Ct      ┆ Ctprime ┆ a       ┆ u4      ┆ v4      ┆ data_ty │
│ ---     ┆ ---      ┆ ---  ┆ ---      ┆ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---     ┆ pe      │
│ f64     ┆ f64      ┆ f64  ┆ f64      ┆ f64     ┆ f64     ┆ f64     ┆ f64     ┆ f64     ┆ ---     │
│         ┆          ┆      ┆          ┆         ┆         ┆         ┆         ┆         ┆ str     │
╞═════════╪══════════╪══════╪══════════╪═════════╪═════════╪═════════╪═════════╪═════════╪═════════╡
│ -2.8712 ┆ 8.589028 ┆ 0.0  ┆ 0.513856 ┆ 0.84543 ┆ 1.74237 ┆ 0.30342 ┆ 0.39315 ┆ 0.0     ┆ constan │
│         ┆          ┆      ┆          ┆ 2       ┆ 8       ┆ 4       ┆ 1       ┆         ┆ t_tsr   │
│ -2.8712 ┆ 8.589028 ┆ 2.5  ┆ 0.513337 ┆ 0.84482 ┆ 1.74313 ┆ 0.30316 ┆ 0.39381 ┆ -0.0092 ┆ constan │
│         ┆          ┆      ┆          ┆ 5       ┆ 4       ┆ 2       ┆ 

# BEM vs AD

In [2]:
from src.generate_over_yaw_Ctprime import generate_yaw_powerloss_data

df = generate_yaw_powerloss_data()
CACHE_FN = cachedir / "yaw_powerloss_data.csv"

df.write_csv(CACHE_FN)

df

100%|██████████| 620/620 [03:52<00:00,  2.67it/s]


rotor,Ctprime,yaw,Ct,Cp,a,u4,v4,Cp_ref,power_loss
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""AD""",0.4,-30.0,0.257668,0.206805,0.073235,0.860985,0.032208,0.300526,0.688142
"""AD""",0.4,-28.0,0.266547,0.217586,0.075468,0.855848,0.031284,0.300526,0.724017
"""AD""",0.4,-26.0,0.274953,0.227959,0.077558,0.850965,0.030133,0.300526,0.758535
"""AD""",0.4,-24.0,0.282853,0.237854,0.079508,0.846358,0.028762,0.300526,0.791459
"""AD""",0.4,-22.0,0.290221,0.247208,0.081311,0.842046,0.02718,0.300526,0.822585
"""AD""",0.4,-20.0,0.297033,0.255962,0.082964,0.838047,0.025398,0.300526,0.851715
"""AD""",0.4,-18.0,0.303265,0.264061,0.084465,0.834378,0.023429,0.300526,0.878662
"""AD""",0.4,-16.0,0.308898,0.271451,0.085812,0.831054,0.021286,0.300526,0.903254
"""AD""",0.4,-14.0,0.313913,0.278089,0.087001,0.828087,0.018986,0.300526,0.92534
